### Library

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
import re
from datetime import datetime
import time
from selenium import webdriver
import calendar 
import numpy as np
from selenium import webdriver
import pandas as pd

### Functions 

In [71]:
'''
Function to get url based on input location, month and year for timeanddate.com
input: locaiton (str), month （int), year (int)
output: string url
'''
def get_url(location, month, year):
    date = {'location': location, 'month': month, 'year': year}
    URL = 'https://www.timeanddate.com/weather/australia/{location}/historic?month={month}&year={year}'.format(**date)
    return URL

'''
Function to automate select in website for dynamic chaning tables by the specified id.
Function will select each option, and for each option extract the table by the specified id, concat all the tables into one
input: instance of selenium.webdriver.chrome.webdriver.WebDriver
output: dataframe of all the extracted table
'''
def get_all_tables(d, month):
    frames = []
    for i in d.find_element_by_id('wt-his-select').find_elements_by_tag_name('option'):
        i.click()
        time.sleep(3)
        df = extract_table(d.page_source, 'wt-his', month) # extract table
        df = fill_in_date(df) # fill in the blank dates rows
        frames.append(df) 

    concated = pd.concat(frames)
    return concated

'''
Function to fill in the empty date rows in date column
input: dataframe
output: dataframe
'''
def fill_in_date(df):
    df['date'][df['date'] == ''] = np.NaN
    df = df.fillna(method='ffill')
    return df

'''
Function to split something like '12:00amSun, 27 Sep' into ['12:00am', '27 Sep']
input: date (str)
output: list with length of 2
'''
def split_time_date(date): 
    day_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    date_split = [date, '']
    
    for day in day_of_week:
        if day in date:
            split_on = day + ', '
            date_split = date.split(split_on)

    return date_split

'''
Function to take in page source, and extract the table 
input: page source, and id of table
output: dataframe of extracted table
'''
def extract_table(_d, table_id, month):
    soup = BeautifulSoup(_d)
    table = soup.find('table', attrs={'id': table_id})

    Data = []
    for tr in table.find('tbody').find_all('tr'):
        dict = {}
        datetime = tr.find('th').text.strip()
        split_datetime = split_time_date(datetime)
        dict['month'] = month
        dict['date'] = split_datetime[1]
        dict['time'] = split_datetime[0]
        #dict['time'] = tr.find('th').text.strip()
        all_td = tr.find_all('td')
        dict['temp'] = all_td[1].text
        dict['weather'] = all_td[2].text
        dict['wind'] = all_td[3].text
        dict['arrow'] = all_td[4].text

        dict['humidity'] = all_td[5].text
        dict['barometer'] = all_td[6].text
        dict['visibility'] = all_td[7].text

        Data.append(dict)
        df = pd.DataFrame(Data)
    
    return df

### Get list of location from txt

In [72]:
path = '../assets/list of locations in vic.txt'
with open(path, 'r', encoding='utf-8') as f:
    # remove newline character
    locations = [x.strip() for x in f.readlines()]

location_list = locations[2:-4]

### Use selenium to automate the select and click on each date

In [ ]:
# this requires browser specific driver: https://www.selenium.dev/downloads/
d = webdriver.Chrome('../chromedriver_win32/chromedriver.exe')

year_list = [2020, 2019, 2018, 2017]
for location in location_list:
    print(location)
    location = location.replace('，', '').lower()
    weather_df_list = []
    for year in year_list:
        if year != 2020:
            for month in range(1, 12+1):
                url = get_url(location, month, year)
                print(url)
                d.get(url)
                month_weather_df = get_all_tables(d, month)
                weather_df_list.append(month_weather_df)
        else:
            for month in range(1, 8+1):
                url = get_url(location, month, year)
                print(url)
                d.get(url)
                month_weather_df = get_all_tables(d, month)
                weather_df_list.append(month_weather_df)
            
            # write to file with location name
            df = pd.concat(weather_df_list)
            output_name = f"../assets/data/{location}_weather.xlsx"
            df.to_excel(output_name, index=False)

Ararat，
https://www.timeanddate.com/weather/australia/ararat/historic?month=1&year=2020


C:\Users\Dango\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [66]:
df

,date,time,temp,weather,wind,arrow,humidity,barometer,visibility
0,28 Sep,10:00 pm,5 °C,Quite cool.,6 km/h,↑,89%,1026 mbar,N/A
1,28 Sep,9:00 pm,5 °C,Quite cool.,7 km/h,↑,86%,1027 mbar,N/A
2,28 Sep,8:00 pm,6 °C,Quite cool.,7 km/h,↑,83%,1027 mbar,N/A
3,28 Sep,7:00 pm,7 °C,Cool.,9 km/h,↑,81%,1026 mbar,N/A
4,28 Sep,6:00 pm,9 °C,Cool.,11 km/h,↑,71%,1026 mbar,N/A
...,...,...,...,...,...,...,...,...,...
21,14 Sep,9:00 pm,7 °C,Cool.,7 km/h,↑,85%,1030 mbar,N/A
22,14 Sep,10:00 pm,8 °C,Cool.,9 km/h,↑,87%,1030 mbar,N/A
23,14 Sep,11:00 pm,8 °C,Cool.,9 km/h,↑,89%,1030 mbar,N/A
0,13 Sep,10:00 pm,11 °C,Cool.,15 km/h,↑,78%,1024 mbar,N/A


### Extract "script" 

Not Used!

In [ ]:
URL = 'https://www.timeanddate.com/weather/australia/ararat/historic?month=9&year=2020'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [114]:
allScripts = soup.find_all("script", type="text/javascript")
dataScript = (allScripts[1].text).strip('\n')

In [120]:
'''
unfortunately this does not work due to changing end bit:
    pattern = re.compile("var data=\{(.*?)\};")
    data = pattern.findall(dataScript)
Function to return strip values at beginning and end
input: month and year
output: begin(constant) and end strip values
'''
def getStripValues(month, year):
    date = {'month': month, 'year': year}
    end = "window.month={month};window.year={year};".format(**date)
    begin = "var data="
    return begin, end

In [121]:
begin, end = getStripValues(9, 2020)

In [141]:
dataString = dataScript.strip(begin).strip(end)
dataJson = json.loads(dataString)
data = dataJson['detail']

In [152]:
# javascript datetime stamp can be convereted using below
datetime.fromtimestamp(1598918400000 / 1000.0)

datetime.datetime(2020, 9, 1, 10, 0)